- [PyTorch for Deep Learning - Full Course (FreeCodeCamp Video)](https://www.youtube.com/watch?v=GIsg-ZUy0MY&t=205s)

In [4]:
%config Completer.use_jedi = False

In [2]:
import torch
import numpy as np

In [3]:
data = [[1,2,3], [5,43,2]]
x_data  = torch.tensor(data)
x_data 

tensor([[ 1,  2,  3],
        [ 5, 43,  2]])

In [4]:
np_array = np.array(data)
x_np = torch.from_numpy(np_array)
np_array, x_np

(array([[ 1,  2,  3],
        [ 5, 43,  2]]),
 tensor([[ 1,  2,  3],
         [ 5, 43,  2]]))

### tensor

In [5]:
x = torch.tensor(3.)
w = torch.tensor(4., requires_grad=True)
b = torch.tensor(5., requires_grad=True)

In [6]:
y = x*w +b
y

tensor(17., grad_fn=<AddBackward0>)

In [8]:
y.backward()

In [12]:
print(x.grad, y.grad, w.grad, b.grad)

None None tensor(3.) tensor(1.)


/home/ubuntu/torchtest/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations.
  """Entry point for launching an IPython kernel.


### numpy

In [2]:
import numpy as np

In [3]:
x = np.array()

TypeError: Required argument 'object' (pos 1) not found

In [6]:
np.ndarray, np.array

(numpy.ndarray, <function numpy.array>)

### apples and oranges

In [18]:
import torch

In [30]:
#X = (temp, rain, humidity)
inputs = np.array([[73,67,43],
     [91,88,64],
     [102,43,37],
     [87,134,58],
     [69,96,70]], dtype='float32')
inputs = torch.from_numpy(inputs)

In [31]:
#Y = (apples, oranges)
outputs = np.array([
     [56,70],
     [81,101],
     [119,133],
     [22,37],
     [103,119]
 ], dtype='float32')
outputs = torch.from_numpy(outputs)

In [40]:
w = torch.randn(2,3,requires_grad=True) # 2 output vars from 3 input vars.. after we transpose it
b = torch.randn(2, requires_grad=True) # initialized with normal distribution with mean 0 and standard deviation 1.
print(w,b)

tensor([[ 0.9792, -0.1531, -0.2063],
        [ 0.5462, -0.4614, -1.7207]], requires_grad=True) tensor([0.9383, 1.0918], requires_grad=True)


In [54]:
#define mathetmatical model
def model(x, w, b):
    return x @ w.t() + b

In [41]:
# see what it predicts
preds = model(inputs)
print(preds)

tensor([[  53.2944,  -63.9394],
        [  63.3736,  -99.9316],
        [  86.6036,  -26.7034],
        [  53.6525, -113.0154],
        [  39.3684, -125.9624]], grad_fn=<AddBackward0>)


In [34]:
# see what it should predict
print(outputs)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [35]:
# see how different it is
def mse(t1, t2):
    diff = t1-t2
    return torch.sum(diff*diff)/diff.numel()

In [50]:
loss = mse(preds, outputs)
print(loss)

tensor(17274.8008, grad_fn=<DivBackward0>)


In [44]:
print(f"w.grad before backpropagation: {w.grad}")
loss.backward()
print(w)
print(f"w.grad after backpropagation: {w.grad}") # df/dw


w.grad before backpropagation: None
tensor([[ 0.9792, -0.1531, -0.2063],
        [ 0.5462, -0.4614, -1.7207]], requires_grad=True)
w.grad after backpropagation: tensor([[ -1348.5496,   -998.5267,  -1012.2916],
        [-14861.1689, -15428.3232, -10075.2607]])


In [45]:
print(f"b.grad before backpropagation: {b.grad}")
loss.backward()
print(b)
print(f"b.grad after backpropagation: {b.grad}") # df/db


# RuntimeError: Trying to backward through the graph a second time, 
# but the saved intermediate results have already been freed. 
# Specify retain_graph=True when calling .backward() or autograd.grad() the first time.


b.grad before backpropagation: tensor([ -16.9415, -177.9104])


RuntimeError: Trying to backward through the graph a second time, but the saved intermediate results have already been freed. Specify retain_graph=True when calling .backward() or autograd.grad() the first time.

In [46]:
w.grad.zero_()
b.grad.zero_()
print(w.grad, b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]]) tensor([0., 0.])


In [56]:
w = torch.randn(2,3,requires_grad=True) # 2 output vars from 3 input vars.. after we transpose it
b = torch.randn(2, requires_grad=True) # initialized with normal distribution with mean 0 and standard deviation 1.
preds = model(inputs, w, b)
loss = mse(preds, outputs)
loss.backward()

# We use torch.no_grad to indicate to PyTorch 
# that we shouldn't track, calculate or modify gradients 
# while updating the weights and biases.
with torch.no_grad():
    learning_rate = 1e-5
    w -= w.grad * learning_rate
    b -= b.grad * learning_rate
    w.grad.zero_()
    b.grad.zero_()

In [57]:
w, w.grad

(tensor([[ 0.2837,  1.9113, -0.8235],
         [ 0.4683,  0.4118, -0.0504]], requires_grad=True),
 tensor([[0., 0., 0.],
         [0., 0., 0.]]))

In [94]:
for i in range(1000):
    preds = model(inputs, w, b)
    loss = mse(preds, outputs)
    if (i % 20 == 0):
        print(loss)
    loss.backward()

    # We use torch.no_grad to indicate to PyTorch 
    # that we shouldn't track, calculate or modify gradients 
    # while updating the weights and biases.
    with torch.no_grad():
        learning_rate = 1e-5
        w -= w.grad * learning_rate
        b -= b.grad * learning_rate
        w.grad.zero_()
        b.grad.zero_()

tensor(166.0341, grad_fn=<DivBackward0>)
tensor(165.9020, grad_fn=<DivBackward0>)
tensor(165.7747, grad_fn=<DivBackward0>)
tensor(165.6521, grad_fn=<DivBackward0>)
tensor(165.5339, grad_fn=<DivBackward0>)
tensor(165.4202, grad_fn=<DivBackward0>)
tensor(165.3106, grad_fn=<DivBackward0>)
tensor(165.2050, grad_fn=<DivBackward0>)
tensor(165.1034, grad_fn=<DivBackward0>)
tensor(165.0053, grad_fn=<DivBackward0>)
tensor(164.9109, grad_fn=<DivBackward0>)
tensor(164.8200, grad_fn=<DivBackward0>)
tensor(164.7323, grad_fn=<DivBackward0>)
tensor(164.6480, grad_fn=<DivBackward0>)
tensor(164.5667, grad_fn=<DivBackward0>)
tensor(164.4884, grad_fn=<DivBackward0>)
tensor(164.4130, grad_fn=<DivBackward0>)
tensor(164.3403, grad_fn=<DivBackward0>)
tensor(164.2702, grad_fn=<DivBackward0>)
tensor(164.2027, grad_fn=<DivBackward0>)
tensor(164.1379, grad_fn=<DivBackward0>)
tensor(164.0752, grad_fn=<DivBackward0>)
tensor(164.0150, grad_fn=<DivBackward0>)
tensor(163.9569, grad_fn=<DivBackward0>)
tensor(163.9009,

In [95]:
print(w, b)

tensor([[ 0.7763, -1.3736,  2.3343],
        [ 0.8697, -1.4122,  2.5698]], requires_grad=True) tensor([ 1.1411, -0.4500], requires_grad=True)


In [96]:
print(preds)

tensor([[ 66.1565,  78.9261],
        [100.3044, 118.8911],
        [107.6321, 122.6227],
        [ 20.0077,  35.0331],
        [ 86.2410, 103.8777]], grad_fn=<AddBackward0>)


In [97]:
print(outputs)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


### Linear Regression using torch.nn

In [130]:
import torch.nn as nn

# allows us to access a small section of the training data using the array indexing notation ([0:3] in the above code). It returns a tuple (or pair), in which the first element contains the input variables for the selected rows, and the second contains the targets.
from torch.utils.data import TensorDataset 

#  split the data into batches of a predefined size while training
from torch.utils.data import DataLoader 

# Import nn.functional - contains many useful loss functions and several other utilities.
import torch.nn.functional as F

In [99]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], [91, 88, 64], [87, 134, 58], 
                   [102, 43, 37], [69, 96, 70], [73, 67, 43], 
                   [91, 88, 64], [87, 134, 58], [102, 43, 37], 
                   [69, 96, 70], [73, 67, 43], [91, 88, 64], 
                   [87, 134, 58], [102, 43, 37], [69, 96, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], [81, 101], [119, 133], 
                    [22, 37], [103, 119], [56, 70], 
                    [81, 101], [119, 133], [22, 37], 
                    [103, 119], [56, 70], [81, 101], 
                    [119, 133], [22, 37], [103, 119]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [101]:
train_ds = TensorDataset(inputs, targets)

In [104]:
train_ds[0:3, 1]

(tensor([ 67.,  88., 134.]), tensor([ 70., 101., 133.]))

In [109]:
inputs.size(), len(inputs)

(torch.Size([15, 3]), 15)

In [110]:
batch_size = 5

# shuffle set to True to have the data reshuffled at every epoch
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [125]:
# shuffled on every iteration
for xbatch, ybatch in train_dl:
    print(xbatch)
    print(ybatch)

tensor([[102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [102.,  43.,  37.]])
tensor([[ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [ 22.,  37.]])
tensor([[ 69.,  96.,  70.],
        [ 91.,  88.,  64.],
        [ 69.,  96.,  70.],
        [102.,  43.,  37.],
        [ 73.,  67.,  43.]])
tensor([[103., 119.],
        [ 81., 101.],
        [103., 119.],
        [ 22.,  37.],
        [ 56.,  70.]])
tensor([[ 87., 134.,  58.],
        [ 87., 134.,  58.],
        [ 87., 134.,  58.],
        [ 91.,  88.,  64.],
        [ 73.,  67.,  43.]])
tensor([[119., 133.],
        [119., 133.],
        [119., 133.],
        [ 81., 101.],
        [ 56.,  70.]])


In [126]:
model = nn.Linear(3,2) # 3 input vars to 2 output vars

In [127]:
print(model.weight, model.bias)
# y = xW^T + b
# shape of W is 2x3

Parameter containing:
tensor([[ 0.1638,  0.2068, -0.3615],
        [ 0.5245, -0.1700,  0.0985]], requires_grad=True) Parameter containing:
tensor([-0.4813, -0.3101], requires_grad=True)


In [128]:
preds = model(inputs)

In [129]:
preds

tensor([[ 9.7897, 30.8231],
        [ 9.4902, 38.7623],
        [20.5184, 28.2525],
        [11.7449, 49.5232],
        [ 5.3723, 26.4540],
        [ 9.7897, 30.8231],
        [ 9.4902, 38.7623],
        [20.5184, 28.2525],
        [11.7449, 49.5232],
        [ 5.3723, 26.4540],
        [ 9.7897, 30.8231],
        [ 9.4902, 38.7623],
        [20.5184, 28.2525],
        [11.7449, 49.5232],
        [ 5.3723, 26.4540]], grad_fn=<AddmmBackward>)

In [135]:
list(filter(lambda x: 'loss' in x, dir(F)))

['cosine_embedding_loss',
 'ctc_loss',
 'gaussian_nll_loss',
 'hinge_embedding_loss',
 'l1_loss',
 'margin_ranking_loss',
 'mse_loss',
 'multi_margin_loss',
 'multilabel_margin_loss',
 'multilabel_soft_margin_loss',
 'nll_loss',
 'poisson_nll_loss',
 'smooth_l1_loss',
 'soft_margin_loss',
 'triplet_margin_loss',
 'triplet_margin_with_distance_loss']

In [136]:
loss = F.mse_loss(model(inputs), targets)
print(loss)

tensor(5168.5991, grad_fn=<MseLossBackward>)


In [140]:
optim = torch.optim.SGD(model.parameters(), lr=1e-5)

In [141]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))
    
    return 

In [143]:
model = nn.Linear(3,2)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)
fit(100, model, F.mse_loss, optimizer, train_dl)

Epoch [10/100], Loss: 101.0640
Epoch [20/100], Loss: 41.2558
Epoch [30/100], Loss: 20.4440
Epoch [40/100], Loss: 38.2093
Epoch [50/100], Loss: 14.2622
Epoch [60/100], Loss: 0.2922
Epoch [70/100], Loss: 2.0149
Epoch [80/100], Loss: 1.1263
Epoch [90/100], Loss: 1.5358
Epoch [100/100], Loss: 1.9914


In [144]:
model(inputs)

tensor([[ 58.1712,  70.5943],
        [ 83.1211, 100.6067],
        [120.6332, 133.9079],
        [ 22.0033,  37.4102],
        [102.6614, 118.7531],
        [ 58.1712,  70.5943],
        [ 83.1211, 100.6067],
        [120.6332, 133.9079],
        [ 22.0033,  37.4102],
        [102.6614, 118.7531],
        [ 58.1712,  70.5943],
        [ 83.1211, 100.6067],
        [120.6332, 133.9079],
        [ 22.0033,  37.4102],
        [102.6614, 118.7531]], grad_fn=<AddmmBackward>)

In [145]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

In [146]:
params = model.parameters()

In [153]:
print(model)
print(model.parameters())
print(model.weight)

print(model.bias)

Linear(in_features=3, out_features=2, bias=True)
<generator object Module.parameters at 0x7f04536d50f8>
Parameter containing:
tensor([[-0.3934,  0.8671,  0.6582],
        [-0.2928,  0.8117,  0.8684]], requires_grad=True)
Parameter containing:
tensor([0.4906, 0.2416], requires_grad=True)


### Snipperts

###### Split data
```python
from torch.utils.data import random_split

train_ds, val_ds = random_split(dataset, [50000, 10000])
len(train_ds), len(val_ds)
```

##### DataLoader
We can now create data loaders to help us load the data in batches. We'll use a batch size of 128.
```python
from torch.utils.data import DataLoader

batch_size = 128

train_loader = DataLoader(train_ds, batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size)
```
We set shuffle=True for the training data loader to ensure that the batches generated in each epoch are different. This randomization helps generalize & speed up the training process. On the other hand, since the validation data loader is used only for evaluating the model, there is no need to shuffle the images.
